# Run this in Google Colab

<a target="_blank" href="https://colab.research.google.com/github/morriswong/hungrydata/blob/main/llm_with_search.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%%capture
%pip install -qU langchain-groq
%pip install -qU langchain
%pip install -qU langchain-community
%pip install -qU duckduckgo-search

In [2]:
import os
from langchain_groq import ChatGroq
from google.colab import userdata

os.environ["GROQ_API_KEY"] = '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'

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [3]:
from langchain.agents import initialize_agent
from langchain_community.tools import Tool, DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain.agents import Tool

ddg_search = DuckDuckGoSearchResults()

tools = [
   Tool(
       name="DuckDuckGo Search",
       func=ddg_search.run,
       description="Useful to browse information from the Internet.",
   )
]

agent = initialize_agent(
   tools, llm, agent="zero-shot-react-description", verbose=True
)

<ipython-input-3-36ae0ca615a1>:15: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [8]:
from langchain.agents import initialize_agent

agent = initialize_agent(
   tools, llm, agent="zero-shot-react-description", verbose=True
)

agent.run(
   "top 10 competitors  of frooti"
)



> Entering new AgentExecutor chain...
To find the top 10 competitors of Frooti, I need to search the internet for a list of companies that compete with Frooti in the beverage industry. I will use the DuckDuckGo Search tool for this.

Action: DuckDuckGo Search
Action Input: "top 10 competitors of Frooti"
Observation: snippet: Parle Agro has 83 competitors. Parle Agro - Brand of multi-category beverages. Raised funding from 1 investor. ... Brand of multi-category beverages. It offers a wide range of brands, including Frooti, Appy Fizz, LMN, Hippo, Bailley, Dhishoom, Bombay 99, and Smoodh. ... Here is the list of Top 10 competitors of Parle Agro, ranked by Tracxn ..., title: Parle Agro - Company Profile - Tracxn, link: https://tracxn.com/d/companies/parle-agro/__DSW2dmVOrvajz_qa1Tt7-uqKLoTJMV1yATV2NGKN4IA, snippet: Their competitors include brands like slice and Frooti. Product Features-This holds 1.2 L of the beverage. It is mango-flavored. ... The top 10 soft drink brands in India inc

'The top 10 competitors of Frooti include Slice, Raw Pressery, Coca-Cola, Pepsi, Thums Up, Sprite, Limca, Fanta, Mountain Dew, Mirinda, and Aquafina.'

In [ ]:

import os
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchResults, Tool
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langchain_core.utils.function_calling import convert_to_openai_function
from typing import Dict, List, Annotated, Sequence, TypedDict, Union
import operator
import json
from langchain_core.tools import Tool
from pydantic import BaseModel, Field

# Set your Groq API key
os.environ["GROQ_API_KEY"] = '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'

# Initialize the LLM
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Initialize DuckDuckGo search tool
ddg_search = DuckDuckGoSearchResults(name="duckduckgo_results_json")

class SearchInput(BaseModel):
    query: str = Field(description="The search query string")

def duckduckgo_search_run(query: str):
    return ddg_search.run(query)

tools = [
    Tool(
        name="duckduckgo_results_json",
        func=duckduckgo_search_run,
        description="Useful to search the internet for information. Input to this tool is a search query.",
        args_schema=SearchInput,
    )
]

# Convert tools to OpenAI function format
functions = [convert_to_openai_function(t) for t in tools]

class AgentState(TypedDict):
    messages: Annotated[Sequence[Union[HumanMessage, AIMessage]], operator.add]
    next: str

def should_continue(state: AgentState):
    messages = state['messages']
    last_message = messages[-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return "call_tool"
    else:
        return "generate_response"

def call_tool(state: AgentState):
    messages = state['messages']
    last_message = messages[-1]
    tool_calls = last_message.tool_calls
    tool_name = tool_calls[0].name
    tool_input = json.loads(tool_calls[0].args)

    for tool in tools:
        if tool.name == tool_name:
            tool_output = tool.func(**tool_input)
            messages.append(AIMessage(content="", tool_calls=tool_calls))
            messages.append(HumanMessage(content=str(tool_output), name=tool_name))
            return {"messages": messages, "next": "continue"}

    return {"messages": messages, "next": "continue"}

def generate_response(state: AgentState):
    messages = state['messages']
    response = llm.invoke(messages)
    return {"messages": messages + [response], "next": "end"}

def create_agent_workflow():
    workflow = StateGraph(AgentState)
    workflow.add_node("agent", lambda state: {"messages": state['messages'] + [llm.invoke(state['messages'], functions=functions)], "next": should_continue(state)})
    workflow.add_node("call_tool", call_tool)
    workflow.add_node("generate_response", generate_response)

    def route(state):
        next_step = state['next']
        return next_step

    workflow.add_node("route", route)
    workflow.add_edge("agent", "route")
    workflow.add_conditional_edges(
        "route",
        lambda state: state,
        {
            "call_tool": "call_tool",
            "generate_response": "generate_response"
        }
    )
    workflow.add_edge("call_tool", "agent")
    workflow.add_edge("generate_response", END)
    workflow.set_entry_point("agent")
    return workflow.compile()

agent_workflow = create_agent_workflow()

inputs = {"messages": [HumanMessage(content="Use the search tool to find the weather in London for the next 3 days.")]}

for output in agent_workflow.stream(inputs):
    for key, value in output.items():
        print(f"Node '{key}':")
        print(value)
    print("\n---\n")